In [1]:
import random, string
import sys, os
import copy
import numpy as np

In [2]:
def read_input_file(file_path):
    """ 
    Ucitava podatke iz fajla iz OR-Library i pravi matricu cena izmedju cvorova.
    http://people.brunel.ac.uk/~mastjjb/jeb/orlib/pmedinfo.html
    """
    
    try:
        with open(file_path, "r") as f:
            first_line = f.readline().strip().split(' ')
            num_vertices = int(first_line[0])
            num_edges = int(first_line[1])
            p = int(first_line[2])
            
            cost_matrix = np.matrix(np.ones((num_vertices, num_vertices)) * np.inf)
            
            for line in f:
                line = line.strip().split(' ')  
                cost_matrix[int(line[0])-1, int(line[1])-1] = int(line[2])
                cost_matrix[int(line[1])-1, int(line[0])-1] = int(line[2])
        
            for i in range(0, num_vertices):
                cost_matrix[i, i] = 0
               
            # Floyd–Warshall algorithm for shortest path
            # https://en.wikipedia.org/wiki/Floyd%E2%80%93Warshall_algorithm
            for k in range(0, num_vertices):
                for i in range(0, num_vertices):
                    for j in range(0, num_vertices):
                        if cost_matrix[i,j] > cost_matrix[i,k] + cost_matrix[k,j]: 
                            cost_matrix[i,j] = cost_matrix[i,k] + cost_matrix[k,j]
                        
        
            return num_vertices, num_edges, p, cost_matrix
        
    except IOError:
        return None
    

In [3]:
num_vertices, num_edges, p, cost_matrix = read_input_file('OR-Library/pmed3.txt')

In [58]:
cost_matrix

matrix([[  0.,  77., 139., ..., 114., 105.,  66.],
        [ 77.,   0.,  62., ..., 191., 182., 143.],
        [139.,  62.,   0., ..., 206., 209., 205.],
        ...,
        [114., 191., 206., ...,   0.,   9.,  48.],
        [105., 182., 209., ...,   9.,   0.,  39.],
        [ 66., 143., 205., ...,  48.,  39.,   0.]])

In [4]:
class Chromosome:
    """
    Klasa Chromosome predstavlja jedan hromozom za koji se cuva njegov genetski kod i 
    vrednost funkcije prilagodjenosti.
    Genetski kod predstavlja potencijalno resenje problema, tj. listu cvorova koji su izabrani 
    kao lokacije za postavljanje objekta.
    """
    def __init__(self, content, fitness):
        self.content = content
        self.fitness = fitness
    def __str__(self): return "%s f=%d" % (self.content, self.fitness)
    def __repr__(self): return "%s f=%d" % (self.content, self.fitness)
    

In [37]:
class GeneticAlgorithm:
   
    def __init__(self,num_facilities, p, cost_matrix, rand_init_population, apply_hypermutation, hypermutation_population_percent=None):
        
        self.num_facilities = num_facilities
        self.p = p
        self.cost_matrix = cost_matrix
        self.rand_init_population = rand_init_population
        self.apply_hypermutation = apply_hypermutation
        self.hypermutation_population_percent = hypermutation_population_percent
    
        self.iterations = 200                             # Maksimalni dozvoljeni broj iteracija
        self.generation_size = 100                        # Broj jedinki u jednoj generaciji
        self.mutation_prob = 0.3                           # Verovatnoca da se desi mutacija
        self.reproduction_size = 70                      # Broj jedinki koji ucestvuje u reprodukciji
        self.current_iteration = 0                         # Koristi se za interno pracenje iteracija algoritma
        self.top_chromosome = None                         # Hromozom koji predstavlja resenje optimizacionog procesa
        self.hypermutation_prob = 0.05
        
    def mutation(self, chromosome):
        """Vrsi mutaciju nad hromozomom sa verovatnocom self.mutation_prob"""
        mp = random.random()
        #print(mp)
        if mp < self.mutation_prob:
            i = random.randint(0, len(chromosome)-1)
            # demand points bez trenutnih medijana:
            demand_points = [element for element in range(0,len(self.cost_matrix)) if element not in chromosome] 
            #print(demand_points)
            chromosome[i] = random.choice(demand_points)
            
        return chromosome
    
    
    def crossover(self, parent1, parent2):
        
        identical_elements = [element for element in parent1 if element in parent2]
        #print(identical_elements)
        
        # Ako su parent1 i parent2 isti, vraca se samo jedan, a drugi se izbacuje iz populacije
        if len(identical_elements) == len(parent1):
            return parent1, None
        
        k = random.randint(1, len(parent1)-len(identical_elements))
        #print(k)
        
        child1 = []
        child2 = []

#         print('Parents:')
#         print(parent1)
#         print(parent2)
        
        exchange_vector_for_parent1 = [element for element in parent1 if element not in identical_elements]
        exchange_vector_for_parent2 = [element for element in parent2 if element not in identical_elements]   
        #print(exchange_vector_for_parent1)
        #print(exchange_vector_for_parent2)
        
        for i in range(k-1):
            exchange_vector_for_parent1[i], exchange_vector_for_parent2[i] = exchange_vector_for_parent2[i], exchange_vector_for_parent1[i]
#             tmp = exchange_vector_for_parent1[i]
#             exchange_vector_for_parent1[i] = exchange_vector_for_parent2[i]
#             exchange_vector_for_parent2[i] = tmp

        #print(exchange_vector_for_parent1)
        #print(exchange_vector_for_parent2)
        
        child1 = identical_elements + exchange_vector_for_parent1
        child2 = identical_elements + exchange_vector_for_parent2
        
        #print(parent1)
        #print(parent2)
#         print('children:')
#         print(child1)
#         print(child2)
#         print('******************')
        
        return child1, child2
    
    def cost_to_nearest_median(self, facility, medians):
        min_cost = self.cost_matrix[facility, medians[0]]
        for median in medians:
            if min_cost > self.cost_matrix[facility, median]:
                min_cost = self.cost_matrix[facility, median]
        return min_cost

    def fitness(self, chromosome):
        #print(self.nearest_median(0, chromosome))
        cost_sum = 0
        for i in range(self.num_facilities):
            cost_sum += self.cost_to_nearest_median(i, chromosome)
            #cost_sum += self.cost_matrix[i, self.nearest_median(i, chromosome)]
        return cost_sum
    
    
    def initial_random_population(self):
        """Generise generation_size nasumicnih jedinki."""
        init_population = []
        for k in range(self.generation_size):
            rand_medians = []
            facilities = list(range(self.num_facilities))
            for i in range(self.p):
                rand_median = random.choice(facilities)
                rand_medians.append(rand_median)
                facilities.remove(rand_median)
            init_population.append(rand_medians)
        init_population = [Chromosome(content, self.fitness(content)) for content in init_population]
        self.top_chromosome = min(init_population, key=lambda chromo: chromo.fitness)
        print("Current top solution: %s" % self.top_chromosome)
        return init_population
    
    
    def selection(self, chromosomes):
        """Ranking-based selection method"""
        #print(chromosomes)
        # Hromozomi se sortiraju u rastucem poretku po vrednosti fitnes funkcije
        chromosomes.sort(key=lambda x: x.fitness)
        #print(chromosomes)
        L = self.reproduction_size
        selected_chromosomes = []
        
        for i in range(self.reproduction_size):
            j = L - np.floor((-1 + np.sqrt(1 + 4*random.uniform(0, 1)*(L**2 + L))) / 2)
            #print(j)
            selected_chromosomes.append(chromosomes[int(j)])
        return selected_chromosomes
    
    
    def create_generation(self, for_reproduction):
        """
        Od jedinki dobijenih u okviru 'for_reproduction' generise novu generaciju
        primenjujuci genetske operatore 'crossover' i 'mutation'.
        Nova generacija je iste duzine kao i polazna.
        """
        new_generation = []
       
        while len(new_generation) < self.generation_size:
            parents = random.sample(for_reproduction, 2)
            child1, child2 = self.crossover(parents[0].content, parents[1].content)

            self.mutation(child1)
            new_generation.append(Chromosome(child1, self.fitness(child1)))
            
            if child2 != None:
                self.mutation(child2)
                new_generation.append(Chromosome(child2, self.fitness(child2)))
            
        return new_generation
    
    
    def nearest_median(self, facility, medians):
        min_cost = self.cost_matrix[facility, medians[0]]
        nearest_med = medians[0]
        for median in medians:
            if min_cost > self.cost_matrix[facility, median]:
                nearest_med = median        
        return nearest_med
    
    
    def initial_population(self):
        """ Generise inicijalnu populaciju. 
            Na osnovu rada: 
                Oksuz, Satoglu, Kayakutlu: 'A Genetic Algorithm for the P-Median Facility Location Problem'
        """
        
        init_population = []
        for k in range(self.generation_size):
            
            # Randomly select p-medians
            medians = []
            facilities = list(range(self.num_facilities))
            for i in range(self.p):
                rand_median = random.choice(facilities)
                medians.append(rand_median)
                facilities.remove(rand_median)
            #print(medians)
                
            # Assign all demand points to nearest median
            median_nearestpoints_map = dict((el, []) for el in medians)
            for i in range(self.num_facilities):
                median_nearestpoints_map[self.nearest_median(i, medians)].append(i)
            #print(median_nearestpoints_map)
                
            n = len(medians)
            for i in range(n):
                median = medians[i]
                # Determine the center point which has minimum distance to all demand points 
                # that assigned this median
                min_dist = float(np.inf)
                center_point = median
                #print(median_nearestpoints_map)
                #demand_points = median_nearestpoints_map[median]
                
                cluster = [median] + median_nearestpoints_map[median]
                for point in cluster:
                    dist = 0
                    for other_point in cluster:
                        dist += self.cost_matrix[point, other_point]
                    if dist < min_dist:
                        min_dist = dist
                        center_point = point
                                        
                # Replace the median with center point
                medians[i] = center_point
            #print(medians)
            #print('********')
            
            #Calculate fitness value for the individual
            init_population.append(medians)
        init_population = [Chromosome(content, self.fitness(content)) for content in init_population]
        self.top_chromosome = min(init_population, key=lambda chromo: chromo.fitness)
        print("Current top solution: %s" % self.top_chromosome)
        return init_population

    
    
    def optimize(self):
        
        if self.rand_init_population:
            chromosomes = self.initial_random_population()
        else:
            chromosomes = self.initial_population()

        while self.current_iteration < self.iterations:
            print("Iteration: %d" % self.current_iteration)

            # Izaberemo iz populacije skup jedinki za reprodukciju
            for_reproduction = self.selection(chromosomes)

            # Primenom operatora ukrstanja i mutacije kreiraj nove jedinke
            # i izracunaj njihovu prilagodjenost.
            # Dobijene jedinke predstavljaju novu generaciju.
            chromosomes = self.create_generation(for_reproduction)
            
            if self.apply_hypermutation:
                hp = random.random()
                if hp < self.hypermutation_prob:
                    print("Hypermutation...")
                    chromosomes_content = [chromo.content for chromo in chromosomes]
                    k = int(self.generation_size * self.hypermutation_population_percent / 100)
                    individuals_subset = random.sample(chromosomes_content, k)
                    
                    for individual in individuals_subset:
                        chromosomes_content.remove(individual)
                    
                    new_individuals_subset = self.hypermutation(individuals_subset)
                    
                    for individual in new_individuals_subset:
                        chromosomes_content.append(individual)
                    
                    chromosomes = [Chromosome(chromo_content, self.fitness(chromo_content)) for chromo_content in chromosomes_content]
                
            
            self.current_iteration += 1
            
            chromosome_with_min_fitness = min(chromosomes, key=lambda chromo: chromo.fitness)
            if chromosome_with_min_fitness.fitness < self.top_chromosome.fitness:
                self.top_chromosome = chromosome_with_min_fitness
            print("Current top solution: %s" % self.top_chromosome)
            print()
            
        print()
        print("Final top solution: %s" % self.top_chromosome)
    
    def hypermutation(self, individuals_subset):
        
        # TODO: Ovaj deo izdvajanja procenta populacije izbaciti u optimize(), pa poslati samo taj deo populacije
        #k = int(self.generation_size * self.hypermutation_population_percent / 100)
#         print(self.generation_size)
#         print(self.hypermutation_population_percent)
#         print(k)
        #individuals_subset = random.sample(chromosomes, k)
#        print(individuals_subset)
        
        n = len(individuals_subset)
        for idx in range(n):
            X = individuals_subset[idx]
            
            # Let H be the set of facility indexes that are not currently present 
            # in the genotype of individual X
            H = [element for element in range(self.num_facilities) if element not in X]
#             print(x)
#             print(h)
#             print('******')
            for i in H:
                best = X
                for j in X:
                    Y = copy.deepcopy(X)
                    Y.remove(j)
                    Y = Y + [i]
                    if self.fitness(Y) < self.fitness(best):
                        best = Y
                        
                if self.fitness(best) < self.fitness(X):
                    individuals_subset[idx] = best
        
        return individuals_subset

    

In [65]:
genetic = GeneticAlgorithm(num_vertices, p, cost_matrix, rand_init_population=False, apply_hypermutation=False)
genetic.optimize()

Current top solution: [25, 73, 65, 23, 89, 8, 80, 97, 47, 54] f=4605
Iteration: 0
Current top solution: [25, 73, 65, 23, 89, 8, 80, 97, 47, 54] f=4605

Iteration: 1
Current top solution: [25, 73, 65, 23, 89, 8, 80, 97, 47, 54] f=4605

Iteration: 2
Current top solution: [25, 73, 65, 23, 89, 8, 80, 97, 47, 54] f=4605

Iteration: 3
Current top solution: [98, 73, 54, 35, 8, 80, 95, 40, 31, 68] f=4509

Iteration: 4
Current top solution: [98, 35, 25, 54, 73, 14, 76, 64, 83, 95] f=4483

Iteration: 5
Current top solution: [98, 35, 25, 54, 73, 14, 76, 64, 83, 95] f=4483

Iteration: 6
Current top solution: [35, 98, 83, 8, 95, 25, 68, 11, 16, 50] f=4464

Iteration: 7
Current top solution: [35, 73, 54, 98, 25, 95, 14, 76, 47, 50] f=4453

Iteration: 8
Current top solution: [35, 73, 98, 76, 25, 95, 14, 2, 19, 83] f=4452

Iteration: 9
Current top solution: [35, 73, 98, 76, 25, 54, 14, 95, 45, 68] f=4362

Iteration: 10
Current top solution: [35, 73, 98, 76, 25, 54, 14, 95, 45, 68] f=4362

Iteration: 1

Current top solution: [35, 25, 54, 98, 14, 73, 47, 68, 8, 43] f=4277

Iteration: 98
Current top solution: [35, 25, 54, 98, 14, 73, 47, 68, 8, 43] f=4277

Iteration: 99
Current top solution: [35, 25, 54, 98, 14, 73, 47, 68, 8, 43] f=4277

Iteration: 100
Current top solution: [35, 25, 54, 98, 14, 73, 47, 68, 8, 43] f=4277

Iteration: 101
Current top solution: [35, 25, 54, 98, 14, 73, 47, 68, 8, 43] f=4277

Iteration: 102
Current top solution: [35, 25, 54, 98, 14, 73, 47, 68, 8, 43] f=4277

Iteration: 103
Current top solution: [35, 25, 54, 98, 14, 73, 47, 68, 8, 43] f=4277

Iteration: 104
Current top solution: [35, 25, 54, 98, 14, 73, 47, 68, 8, 43] f=4277

Iteration: 105
Current top solution: [35, 25, 54, 98, 14, 73, 47, 68, 8, 43] f=4277

Iteration: 106
Current top solution: [35, 25, 54, 98, 14, 73, 47, 68, 8, 43] f=4277

Iteration: 107
Current top solution: [35, 25, 54, 98, 14, 73, 47, 68, 8, 43] f=4277

Iteration: 108
Current top solution: [35, 25, 54, 98, 14, 73, 47, 68, 8, 43] f=427

Current top solution: [47, 98, 76, 35, 14, 25, 54, 73, 68, 20] f=4269

Iteration: 194
Current top solution: [47, 98, 76, 35, 14, 25, 54, 73, 68, 20] f=4269

Iteration: 195
Current top solution: [47, 98, 76, 35, 14, 25, 54, 73, 68, 20] f=4269

Iteration: 196
Current top solution: [47, 98, 76, 35, 14, 25, 54, 73, 68, 20] f=4269

Iteration: 197
Current top solution: [47, 98, 76, 35, 14, 25, 54, 73, 68, 20] f=4269

Iteration: 198
Current top solution: [47, 98, 76, 35, 14, 25, 54, 73, 68, 20] f=4269

Iteration: 199
Current top solution: [47, 98, 76, 35, 14, 25, 54, 73, 68, 20] f=4269


Final top solution: [47, 98, 76, 35, 14, 25, 54, 73, 68, 20] f=4269


In [66]:
genetic2 = GeneticAlgorithm(num_vertices, p, cost_matrix, rand_init_population=True, apply_hypermutation=False)
genetic2.optimize()

Current top solution: [66, 51, 13, 94, 25, 50, 36, 84, 76, 73] f=5063
Iteration: 0
Current top solution: [66, 51, 13, 94, 25, 50, 36, 84, 76, 73] f=5063

Iteration: 1
Current top solution: [53, 47, 23, 14, 98, 20, 13, 38, 8, 92] f=4796

Iteration: 2
Current top solution: [53, 47, 23, 14, 98, 20, 13, 38, 8, 92] f=4796

Iteration: 3
Current top solution: [53, 47, 23, 14, 98, 20, 13, 38, 8, 92] f=4796

Iteration: 4
Current top solution: [53, 47, 23, 14, 98, 20, 13, 38, 8, 92] f=4796

Iteration: 5
Current top solution: [56, 14, 94, 27, 47, 20, 84, 8, 98, 25] f=4661

Iteration: 6
Current top solution: [56, 14, 94, 27, 47, 20, 84, 8, 98, 25] f=4661

Iteration: 7
Current top solution: [56, 14, 94, 27, 47, 20, 84, 8, 98, 25] f=4661

Iteration: 8
Current top solution: [35, 13, 86, 47, 20, 51, 3, 61, 19, 31] f=4599

Iteration: 9
Current top solution: [35, 47, 9, 99, 13, 32, 86, 53, 68, 31] f=4570

Iteration: 10
Current top solution: [35, 47, 9, 99, 13, 32, 86, 53, 68, 31] f=4570

Iteration: 11
C

Current top solution: [53, 35, 13, 47, 86, 68, 98, 9, 31, 56] f=4381

Iteration: 99
Current top solution: [53, 35, 13, 47, 86, 68, 98, 9, 31, 56] f=4381

Iteration: 100
Current top solution: [53, 35, 13, 47, 86, 68, 98, 9, 31, 56] f=4381

Iteration: 101
Current top solution: [53, 35, 13, 47, 86, 68, 98, 9, 31, 56] f=4381

Iteration: 102
Current top solution: [53, 35, 13, 47, 86, 68, 98, 9, 31, 56] f=4381

Iteration: 103
Current top solution: [53, 35, 13, 47, 86, 68, 98, 9, 31, 56] f=4381

Iteration: 104
Current top solution: [53, 35, 13, 47, 86, 68, 98, 9, 31, 56] f=4381

Iteration: 105
Current top solution: [53, 31, 9, 47, 73, 98, 13, 68, 35, 11] f=4340

Iteration: 106
Current top solution: [53, 31, 9, 47, 73, 98, 13, 68, 35, 11] f=4340

Iteration: 107
Current top solution: [53, 31, 9, 47, 73, 98, 13, 68, 35, 11] f=4340

Iteration: 108
Current top solution: [53, 31, 9, 47, 73, 98, 13, 68, 35, 11] f=4340

Iteration: 109
Current top solution: [53, 31, 9, 47, 73, 98, 13, 68, 35, 11] f=43

Current top solution: [31, 47, 98, 8, 68, 35, 86, 73, 53, 13] f=4332

Iteration: 196
Current top solution: [31, 47, 98, 8, 68, 35, 86, 73, 53, 13] f=4332

Iteration: 197
Current top solution: [31, 47, 98, 8, 68, 35, 86, 73, 53, 13] f=4332

Iteration: 198
Current top solution: [31, 47, 98, 8, 68, 35, 86, 73, 53, 13] f=4332

Iteration: 199
Current top solution: [31, 47, 98, 8, 68, 35, 86, 73, 53, 13] f=4332


Final top solution: [31, 47, 98, 8, 68, 35, 86, 73, 53, 13] f=4332


In [38]:
genetic3 = GeneticAlgorithm(num_vertices, p, cost_matrix, rand_init_population = True, apply_hypermutation=True, hypermutation_population_percent=10)
genetic3.optimize()

Current top solution: [25, 87, 58, 6, 30, 8, 84, 95, 44, 16] f=5116
Iteration: 0
Current top solution: [25, 87, 58, 6, 30, 8, 84, 95, 44, 16] f=5116

Iteration: 1
Current top solution: [32, 61, 53, 89, 0, 84, 40, 95, 13, 4] f=4994

Iteration: 2
Current top solution: [32, 61, 53, 89, 0, 84, 40, 95, 13, 4] f=4994

Iteration: 3
Current top solution: [25, 23, 16, 79, 40, 88, 98, 68, 95, 36] f=4939

Iteration: 4
Current top solution: [76, 25, 0, 44, 53, 89, 23, 60, 86, 43] f=4815

Iteration: 5
Current top solution: [76, 25, 0, 44, 53, 89, 23, 60, 86, 43] f=4815

Iteration: 6
Current top solution: [76, 25, 0, 44, 53, 89, 23, 60, 86, 43] f=4815

Iteration: 7
Current top solution: [25, 23, 43, 85, 76, 60, 40, 97, 56, 18] f=4744

Iteration: 8
Current top solution: [25, 76, 23, 95, 43, 50, 97, 85, 56, 69] f=4718

Iteration: 9
Current top solution: [25, 32, 61, 95, 0, 53, 15, 35, 11, 69] f=4704

Iteration: 10
Current top solution: [25, 23, 76, 0, 5, 97, 43, 95, 53, 60] f=4607

Iteration: 11
Curre

Current top solution: [23, 25, 98, 73, 15, 43, 76, 19, 47, 65] f=4347

Iteration: 98
Current top solution: [23, 25, 98, 73, 15, 43, 76, 19, 47, 65] f=4347

Iteration: 99
Current top solution: [23, 25, 98, 73, 15, 43, 76, 19, 47, 65] f=4347

Iteration: 100
Current top solution: [23, 25, 98, 73, 15, 43, 76, 19, 47, 65] f=4347

Iteration: 101
Current top solution: [23, 25, 98, 73, 15, 43, 76, 19, 47, 65] f=4347

Iteration: 102
Current top solution: [23, 25, 98, 73, 15, 43, 76, 19, 47, 65] f=4347

Iteration: 103
Current top solution: [23, 25, 98, 73, 15, 43, 76, 19, 47, 65] f=4347

Iteration: 104
Current top solution: [23, 25, 98, 73, 15, 43, 76, 19, 47, 65] f=4347

Iteration: 105
Current top solution: [23, 25, 98, 73, 15, 43, 76, 19, 47, 65] f=4347

Iteration: 106
Current top solution: [23, 25, 98, 73, 15, 43, 76, 19, 47, 65] f=4347

Iteration: 107
Current top solution: [23, 25, 98, 73, 15, 43, 76, 19, 47, 65] f=4347

Iteration: 108
Current top solution: [23, 25, 98, 73, 15, 43, 76, 19, 4

Current top solution: [25, 98, 13, 54, 76, 95, 68, 73, 35, 20] f=4264

Iteration: 192
Current top solution: [25, 98, 13, 54, 76, 95, 68, 73, 35, 20] f=4264

Iteration: 193
Current top solution: [25, 98, 13, 54, 76, 95, 68, 73, 35, 20] f=4264

Iteration: 194
Current top solution: [25, 98, 13, 54, 76, 95, 68, 73, 35, 20] f=4264

Iteration: 195
Current top solution: [25, 98, 13, 54, 76, 95, 68, 73, 35, 20] f=4264

Iteration: 196
Current top solution: [54, 73, 8, 98, 35, 20, 95, 25, 14, 68] f=4255

Iteration: 197
Current top solution: [54, 73, 8, 98, 35, 20, 95, 25, 14, 68] f=4255

Iteration: 198
Current top solution: [54, 73, 8, 98, 35, 20, 95, 25, 14, 68] f=4255

Iteration: 199
Current top solution: [54, 73, 8, 98, 35, 20, 95, 25, 14, 68] f=4255


Final top solution: [54, 73, 8, 98, 35, 20, 95, 25, 14, 68] f=4255


In [24]:
genetic = GeneticAlgorithm(num_vertices, p, cost_matrix, rand_init_population = False, hypermutation_population_percent = 10)
#genetic.mutation([1,23,47,80])
#genetic.crossover([24,12,9,26,18,40], [8,13,18,36,24,20])
#genetic.crossover([1,23,47,80,40,65,77], [2,50, 23,65,67,89,99])   
#genetic.crossover([1,2,3,4], [1,2,3,4])
#genetic.fitness([65,50,90])
init_pop = genetic.initial_random_population()
#init_pop
#init_pop2 = genetic.initial_population()

Current top solution: [25, 35, 29, 95, 65, 67, 9, 42, 55, 82] f=5057


In [25]:
for_reproduction = genetic.selection(init_pop)
#for_reproduction

In [39]:
new_generation = genetic.create_generation(for_reproduction)
#new_generation

In [27]:
for_hypermutation = [chromo.content for chromo in new_generation]
#print(for_hypermutation)
genetic.hypermutation(for_hypermutation)

[[73, 96, 84, 66, 72, 16, 30, 61, 42, 99],
 [28, 84, 57, 70, 34, 25, 32, 47, 21, 99],
 [96, 84, 66, 72, 31, 73, 10, 33, 19, 99],
 [60, 20, 57, 47, 8, 75, 66, 98, 73, 99],
 [77, 20, 34, 95, 40, 25, 1, 59, 19, 99],
 [8, 85, 96, 94, 54, 77, 64, 24, 20, 99],
 [92, 55, 11, 36, 68, 77, 25, 0, 61, 99],
 [71, 79, 47, 72, 98, 43, 69, 73, 59, 99],
 [11, 94, 61, 22, 39, 47, 88, 36, 23, 99],
 [87, 34, 8, 43, 66, 60, 98, 73, 46, 99]]

In [41]:
num_vertices, num_edges, p, cost_matrix = read_input_file('test.txt')

In [42]:
cost_matrix

matrix([[  0.,  80., 140., 230., 250.],
        [200.,   0.,  60., 150., 170.],
        [140., 150.,   0.,  90., 110.],
        [ 50.,  60., 120.,   0.,  20.],
        [240.,  40., 100., 190.,   0.]])